In [2]:
pip install PyPortfolioOpt

     |████████████████████████████████| 61 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 880 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 48.1 MB/s  eta 0:00:01
     |████████████████████████████████| 249 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 11.4 MB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 23.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [44]:
#using pyfolio 
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import pandas as pd
path = '/Users/anubhanagar/Desktop/APlusBernstein-Project/data/pred/lstm_preds.csv'
df_all = pd.read_csv(path)
df = df_all.iloc[:, -3:]
df

,sc_1d_fwd_rel_ret,mom_1d_fwd_rel_ret,value_1d_fwd_rel_ret
0,0.005416,-0.000329,0.002091
1,-0.005597,-0.000251,0.000522
2,0.002269,0.002999,0.000093
3,0.001520,0.007639,0.002141
4,-0.000087,0.001010,0.002480
...,...,...,...
1841,0.002553,-0.003195,-0.001594
1842,-0.010477,-0.011423,-0.000088
1843,-0.006402,-0.000410,-0.002580
1844,0.002143,-0.009954,0.005957


In [45]:
df["sc_1d_fwd_rel_ret"].isnull().sum()

0

In [46]:
#df.dropna(subset = ["sc_1d_fwd_rel_ret"])
mu = mean_historical_return(df, compounding = False, returns_data = True)
S = CovarianceShrinkage(df).ledoit_wolf()

In [47]:
mu

sc_1d_fwd_rel_ret       0.000780
mom_1d_fwd_rel_ret      0.042487
value_1d_fwd_rel_ret   -0.002527
dtype: float64

In [48]:
S

,sc_1d_fwd_rel_ret,mom_1d_fwd_rel_ret,value_1d_fwd_rel_ret
sc_1d_fwd_rel_ret,4.082278e+06,-0.000000e+00,0.000000e+00
mom_1d_fwd_rel_ret,-0.000000e+00,4.082278e+06,0.000000e+00
value_1d_fwd_rel_ret,0.000000e+00,0.000000e+00,4.082278e+06


In [49]:
#Mean variance optimization
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

In [50]:
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.txt")  # saves to file
print(cleaned_weights)

OrderedDict([('sc_1d_fwd_rel_ret', 0.0), ('mom_1d_fwd_rel_ret', 1.0), ('value_1d_fwd_rel_ret', 0.0)])


In [51]:
ef.portfolio_performance(verbose=True)

Expected annual return: 4.2%
Annual volatility: 202046.5%
Sharpe Ratio: 0.00


(0.042487064970933686, 2020.464857494151, 1.1129649143624753e-05)